In [ ]:
import pymongo
import pandas as pd

data_mongo= pymongo.MongoClient('localhost:27017')

In [2]:
db= data_mongo['test']
rows=db['datarows']

In [3]:
drow1= pd.json_normalize(list(rows.find()[:150000]),sep='_')
drow2= pd.json_normalize(list(rows.find()[150000:300000]),sep='_')

In [4]:
drow3= pd.json_normalize(list(rows.find()[300000:450000]),sep='_')
drow4= pd.json_normalize(list(rows.find()[450000:600000]),sep='_')

In [5]:
drow5= pd.json_normalize(list(rows.find()[600000:750000]),sep='_')
drow6= pd.json_normalize(list(rows.find()[750000:900000]),sep='_')

In [6]:
drow7= pd.json_normalize(list(rows.find()[900000:1050000]),sep='_')
drow8= pd.json_normalize(list(rows.find()[1050000:1200000]),sep='_')
drow9= pd.json_normalize(list(rows.find()[1200000:]),sep='_')

In [7]:
df_row=pd.concat([drow1,drow2,drow3,drow4,drow5,drow6,drow7,drow8,drow9],axis=0)
df_row.shape

(1296780, 22)

In [8]:
df_row._id=df_row._id.astype(str)

In [1]:
import pandas as pd
df= pd.read_csv('data_clean_gps.csv',index_col='Unnamed: 0')
df.createdAt=pd.to_datetime(df.createdAt) 
df.updatedAt=pd.to_datetime(df.updatedAt) 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321019 entries, 1 to 96773
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   UUID                  321019 non-null  object        
 1   dataRowType           321019 non-null  object        
 2   createdAt             321019 non-null  datetime64[ns]
 3   updatedAt             321019 non-null  datetime64[ns]
 4   dataRowData_lat       319620 non-null  float64       
 5   dataRowData_lng       319622 non-null  float64       
 6   dataRowData_gpsAlt    320871 non-null  float64       
 7   dataRowData_gpsVel    320871 non-null  float64       
 8   dataRowData_gpsFixed  320871 non-null  float64       
dtypes: datetime64[ns](2), float64(5), object(2)
memory usage: 24.5+ MB


In [9]:
df_row.dataRowType.unique()

array(['BEACON', 'GPS', 'BATTERY', nan], dtype=object)

In [10]:
data_gps=df_row[df_row.dataRowType=='GPS']
data_beacon= df_row[df_row.dataRowType=='BEACON']
data_battery= df_row[df_row.dataRowType=='BATTERY']

In [ ]:
df_gps=data_gps[['UUID','dataRowType','createdAt','updatedAt','dataRowData_lat','dataRowData_lng','dataRowData_gpsAlt','dataRowData_gpsVel','dataRowData_gpsFixed']]

In [11]:
df_bate=data_battery[['UUID','dataRowType','createdAt','updatedAt','dataRowData_timestamp','dataRowData_battery']]

In [12]:
df_beacon=data_beacon[['UUID','dataRowType','createdAt','updatedAt','dataRowData_timestamp','dataRowData_mac','dataRowData_battery','dataRowData_temperature','dataRowData_rssi','dataRowData_accelerometer']]

In [2]:
def data_devices(data,uuid):
    data=data[data.UUID==uuid]
    data.drop(data[data.dataRowData_lat.isna()].index,inplace=True)
    data.reset_index()
    return data

def gps_data(data):
    gps= data[['dataRowData_lat','dataRowData_lng']]
    gps = gps.dropna()
    return gps

In [ ]:
vaca_1=data_devices(df,'0004A30B00F82E81')

In [ ]:
y= data_devices(df,'0004A30B00F860CC')
y.head()

In [5]:
d_p=vaca_1[['UUID','createdAt','dataRowData_lat','dataRowData_lng']]

In [6]:
from geopy.distance import geodesic,great_circle
from geopy import Point


cordena1=tuple(d_p.iloc[0][['dataRowData_lat','dataRowData_lng']].values)
cordena2= tuple(d_p.iloc[-1][['dataRowData_lat','dataRowData_lng']].values)

dista_km= great_circle(cordena1,cordena2).kilometers
velocidad= vaca_1.dataRowData_gpsVel.mean()
time_travel= dista_km/velocidad

print(f'distancia:{dista_km:.2f} km')
print(f'Velocidad: {velocidad:.2f} km')
print(f'Tiempo:{time_travel:.2f} km')

distancia:0.63 km
Velocidad: 0.40 km
Tiempo:1.57 km


In [8]:
def distancia_recorrida(data):
    cordena1=tuple(data.iloc[0][['dataRowData_lat','dataRowData_lng']].values)
    cordena2= tuple(data.iloc[-1][['dataRowData_lat','dataRowData_lng']].values)
    dista_km= great_circle(cordena1,cordena2).kilometers
    return dista_km

In [9]:
def interview_vaca(data): # tratar de filtrar por perimetro porque si hay valores (que los hay)fuera de rango de los -90 90 da error
    data_dis=[]
    data_vel=[]
    data_time=[]
    for i in range(0,data.shape[0]+1):
        try:
            dista_km= great_circle(tuple(data.iloc[i][['dataRowData_lat','dataRowData_lng']].values),tuple(data.iloc[i+1][['dataRowData_lat','dataRowData_lng']].values)).kilometers
            if dista_km <= 8.:
                data_dis.append(dista_km)
            if data.iloc[i].dataRowData_gpsVel:
                data_vel.append(data.iloc[i].dataRowData_gpsVel)
                data_time.append(dista_km/data.iloc[i].dataRowData_gpsVel)
            else:
                data_time.append(dista_km/pd.Series(data_vel).mean())# les puede dar error si el array de velocidad esta vacio... toma el valor promedio de las velocidades que hay hasta el momento
        except IndexError:
            pass
    return data_dis,data_vel,data_time


In [48]:
vaca_1.iloc[0][['dataRowData_lat','dataRowData_lng']].values

array([-34.19883983333333, -64.04658233333333], dtype=object)

In [10]:
import geopandas as gpd
from shapely.geometry import Point
import folium

In [11]:
setle_lat=-34.163851
setle_lng=-64.071678

gdf= gpd.GeoDataFrame(df,crs='EPSG:4326',geometry=gpd.points_from_xy(df.dataRowData_lng,df.dataRowData_lat))

In [12]:
import math
def perimetro_aprox(hectarea):
    hect=hectarea
    lado = math.sqrt(hect)*10
    perim = lado*4
    return perim

def area_perimetro(latitud,longitud,hectareas):
    setle_lat=latitud
    setle_lng=longitud
    punto_referencia= Point(setle_lng,setle_lat)	
    per_kilo= perimetro_aprox(hectareas)
    circulo= punto_referencia.buffer(per_kilo/111.32) # valor 1 grado aprox en kilometro en el ecuador 
    on_perimetro= gdf[gdf.geometry.within(circulo)]
    return on_perimetro

In [13]:
def filter_area_perimetro(data,latitud,longitud,hectareas):
    gdf= gpd.GeoDataFrame(data,crs='EPSG:4326',geometry=gpd.points_from_xy(data.dataRowData_lng,data.dataRowData_lat))
    setle_lat=latitud
    setle_lng=longitud
    punto_referencia= Point(setle_lng,setle_lat)	
    per_kilo= perimetro_aprox(hectareas)
    circulo= punto_referencia.buffer(per_kilo/111.32) # valor 1 grado aprox en kilometro en el ecuador 
    on_perimetro= gdf[gdf.geometry.within(circulo)]
    return on_perimetro

In [14]:
import string
import random
import seaborn as sns
import matplotlib.colors as mcolors

def random_color():
    """Genera un color cálido aleatorio en formato hexadecimal."""
    # n = random.randint(2,100)
    # paleta= sns.color_palette("deep",n_colors=n)
    # colores_hex = [mcolors.rgb2hex(color) for color in paleta]
    # return colores_hex[random.randint(1,n-1)]
    while True:
        color ='#' + ''.join(random.choices(string.hexdigits[:-6], k=6))
        # Comprueba si el color es cálido
        r, g, b = int(color[1:3], 16), int(color[3:5], 16), int(color[5:7], 16)
        if r > g and r > b and abs(r-g) < 60:
            return color

In [15]:
def grafica_gps(datos_gps,graf,color):
    for i , d in datos_gps.iterrows():
        folium.Marker(location=[d['dataRowData_lat'],d['dataRowData_lng']]).add_to(graf)
    folium.PolyLine(locations=datos_gps,color=color).add_to(graf) #necesitya una funcion que me genere colores aleatorio para identificar

In [ ]:
on_perimetro= area_perimetro(latitud=-26.859657,longitud=-65.245558,hectareas= 11.33)

In [41]:
uuid_devis= on_perimetro.UUID.unique()
uuid_devis

array([], dtype=object)

In [ ]:
aux=0
mp=folium.Map()
for i in uuid_devis[:4]:
    if aux == 0:
        dta=data_devices(on_perimetro, i )
        dta_gps= gps_data(dta)
        mp=folium.Map(location=[setle_lat,setle_lng],zoom_start=15)
        grafica_gps(dta_gps,mp)
        aux = 1
    else:
        dta=data_devices(on_perimetro, i )
        dta_gps= gps_data(dta)
        grafica_gps(dta_gps,mp)
        aux = 1
mp

In [56]:
import datetime

def obtener_fecha_inicio_fin(semana):
    """
    Función que recibe una semana en formato de fecha y devuelve la fecha de inicio y finalización de esa semana.
    
    Args:
    semana (str o datetime): Semana en formato de fecha. Debe estar en formato 'YYYY-MM-DD'.
    
    Returns:
    fecha_inicio (str): Fecha de inicio de la semana en formato 'YYYY-MM-DD'.
    fecha_fin (str): Fecha de finalización de la semana en formato 'YYYY-MM-DD'.
    """
    
    if isinstance(semana, str):
        semana = datetime.datetime.strptime(semana, '%Y-%m-%d')
        
    dia_semana = semana.weekday()
    
    fecha_inicio = semana - datetime.timedelta(days=dia_semana)
    fecha_fin = fecha_inicio + datetime.timedelta(days=6)
    
    fecha_inicio = fecha_inicio.strftime('%Y-%m-%d')
    fecha_fin = fecha_fin.strftime('%Y-%m-%d')
    return fecha_inicio, fecha_fin


def filter_time_day(data,momento):
    switch_dict={
        'noche': data[((data.updatedAt.dt.hour > 20) & (data.updatedAt.dt.hour < 24))| ((data.updatedAt.dt.hour > 0) & (data.updatedAt.dt.hour < 7))],
        'madrugada':  data[(data.updatedAt.dt.hour > 0) & (data.updatedAt.dt.hour < 7)],
        'tarde':data[(data.updatedAt.dt.hour > 13) & (data.updatedAt.dt.hour < 20)],
        'mañana':data[(data.updatedAt.dt.hour > 7) & (data.updatedAt.dt.hour < 13)]
    }
    return switch_dict.get(momento,'valor')


def filter_week_data(data,fecha):
    if isinstance(fecha,int):
        data = data[data.createdAt.dt.week == fecha]
    else:
        week = obtener_fecha_inicio_fin(fecha)
        data = data[(data.createdAt >= week[0]) & (data.createdAt <= week[1])]
    return data

In [46]:
def time_min_max_data(data):
    """Funcion que devuelve los valores de fecha maxima que aparece en los dataframe

    Args:
        data (dataframe): recibe el dataframe saca fecha maxima y fecha minima

    Returns:
        tuple:(datatime,datatime) 
    """
    fecha_max=vaca_1.createdAt.dt.date.max().strftime('%Y-%m-%d')
    fecha_min=vaca_1.createdAt.dt.date.min().strftime('%Y-%m-%d')
    return fecha_min ,fecha_max

In [120]:
def dataframe_interview_vaca(data): # tratar de filtrar por perimetro porque si hay valores (que los hay)fuera de rango de los -90 90 da error
    data_dis=[]
    data_vel=[]
    data_time=[]
    data_inter= []
    for i in range(0,data.shape[0]+1):
        try:
            dista_km= great_circle(tuple(data.iloc[i][['dataRowData_lat','dataRowData_lng']].values),tuple(data.iloc[i+1][['dataRowData_lat','dataRowData_lng']].values)).kilometers
            interval= int(data.iloc[i][['createdAt']].values[0].strftime('%H')) - int(data.iloc[i+1][['createdAt']].values[0].strftime('%H'))
            data_inter.append(interval)
            if dista_km <= 8.:
                data_dis.append(round(dista_km,3))
            if data.iloc[i].dataRowData_gpsVel:
                data_vel.append(round(data.iloc[i].dataRowData_gpsVel,3))
                data_time.append(round(dista_km/data.iloc[i].dataRowData_gpsVel,3))
            else:
                data_time.append(round(dista_km/pd.Series(data_vel).mean().round(3),3))# les puede dar error si el array de velocidad esta vacio... toma el valor promedio de las velocidades que hay hasta el momento
        except IndexError:
            pass
    df = list(zip(data_inter,data_dis,data_vel,data_time))
    df = pd.DataFrame(df,columns=['interval_time','distancia','velocidad','tiempo']) 
    return df

In [115]:
x=filter_time_day(vaca_1,'noche')

In [116]:
x= filter_week_data(x,'2023-02-17')
x = x.sort_values('createdAt')

In [117]:
c=dataframe_interview_vaca(x)

In [72]:
int(x.iloc[0][['createdAt']].values[0].strftime('%H'))

9

In [47]:
time_min_max_data(df)

('2023-02-17', '2023-03-22')

In [118]:
d=list(zip(c[0],c[1],c[2],c[3]))

In [119]:
new_fd=pd.DataFrame(d,columns=['interval_time','distancia','velocidad','tiempo'])
new_fd


,interval_time,distancia,velocidad,tiempo
0,0,0.045,1.475,0.030
1,-3,0.032,0.069,0.459
2,1,0.095,0.025,3.809
3,-2,0.082,0.025,3.274
4,-16,0.065,0.174,0.375
5,-1,0.042,0.125,0.332
6,-1,0.064,0.103,0.625


In [20]:
def graf_muuu(list_vacas,posicion_initial,data):# graficas con folium
    colores=[]
    aux=0
    mp=folium.Map()
    for i in list_vacas:
        color = random_color()
        colores.append(color)
        while color in colores:
            color =random_color()
        if aux == 0:
            dta=data_devices(data, i )
            dta_gps= gps_data(dta)
            mp=folium.Map(location=posicion_initial,zoom_start=15)
            grafica_gps(dta_gps,mp,color)
            aux = 1
        else:
            dta=data_devices(on_perimetro, i )
            dta_gps= gps_data(dta)
            grafica_gps(dta_gps,mp,color)
            aux = 1
    return mp

In [ ]:
graf_muuu(uuid_devis[:6],[setle_lat,setle_lng],d)

In [31]:
import plotly.graph_objects as go

mapbox_access_token = 'pk.eyJ1IjoibmVzdG9yMTYwOCIsImEiOiJjbGc5b2J2d3gwOHgwM2xwamd3cGE4cmExIn0.bPWyeRa73WyNqm1nBNJOvQ' 

def uni_graf(data,color,fig):

    fig.add_trace(go.Scattermapbox(
        lat=data.dataRowData_lat.values,
        lon=data.dataRowData_lng.values,
        mode='lines+markers',
        line=dict(
            width=2,
            color=color,
        ),
        marker=go.scattermapbox.Marker(
            size=8,
            color=color,
            symbol='circle'
        ),
    ))
    return fig

def grafic_map(data,list_vacas,lat_orig,lng_orig,fig):
    colores=[]
    for i in list_vacas:
        color = random_color()
        colores.append(color)
        while color in colores:
            color =random_color()
        dta=data_devices(data, i )
        dta_gps= gps_data(dta)
        uni_graf(dta_gps,color,fig)
        
    fig.update_layout(
        mapbox=dict(
            style='satellite', 
            accesstoken=mapbox_access_token,
            zoom=14, 
            center=dict(lat=lat_orig,lon=lng_orig),
        ),
        showlegend=False
    )
    return fig

In [ ]:
fig = go.Figure()

uni_graf(d_p, random_color(),fig)

fig.update_layout(
    mapbox=dict(
        style='satellite', # Estilo de mapa satelital
        accesstoken=mapbox_access_token,
        zoom=12, # Nivel de zoom inicial del mapa
        center=dict(lat=d_p.iloc[0].dataRowData_lat,lon=d_p.iloc[0].dataRowData_lng),
    ),
    showlegend=False
)

fig.show()

In [ ]:

fig = go.Figure()

# Agregar los puntos al mapa
fig.add_trace(go.Scattermapbox(
    lat=d_p.dataRowData_lat.values,
    lon=d_p.dataRowData_lng.values,
    mode='lines+markers',
    line=dict(
        width=2,
        color='blue',
    ),
    marker=go.scattermapbox.Marker(
        size=8,
        color='red',
        symbol='circle'
    ),
))

# Configurar el estilo del mapa de Mapbox
fig.update_layout(
    mapbox=dict(
        style='satellite', # Estilo de mapa satelital
        accesstoken=mapbox_access_token,
        zoom=12, # Nivel de zoom inicial del mapa
        center=dict(lat=d_p.iloc[0].dataRowData_lat,lon=d_p.iloc[0].dataRowData_lng),
    ),
    showlegend=False
)

# Mostrar el mapa
fig.show()

In [119]:
semana = '2022-08-01'  # Semana en formato 'YYYY-MM-DD'
fecha_inicio, fecha_fin = obtener_fecha_inicio_fin(semana)
print('Fecha de inicio de la semana:', fecha_inicio)
print('Fecha de finalización de la semana:', fecha_fin)

Fecha de inicio de la semana: 2022-08-01
Fecha de finalización de la semana: 2022-08-07
